In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v3 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y,1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
                  
    parameters = {
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    
    return parameters

In [5]:
parameters = initialize_parameters(3,2,1)
parameters


{'W1': array([[ 0.01624345, -0.00611756, -0.00528172],
        [-0.01072969,  0.00865408, -0.02301539]]), 'b1': array([[0.],
        [0.]]), 'W2': array([[ 0.01744812, -0.00761207]]), 'b2': array([[0.]])}

In [6]:
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters["W" + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters["b" + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [7]:
parameters = initialize_parameters_deep([5,4,3])
print(parameters)

{'W1': array([[ 0.00319039, -0.0024937 ,  0.01462108, -0.02060141, -0.00322417],
       [-0.00384054,  0.01133769, -0.01099891, -0.00172428, -0.00877858],
       [ 0.00042214,  0.00582815, -0.01100619,  0.01144724,  0.00901591],
       [ 0.00502494,  0.00900856, -0.00683728, -0.0012289 , -0.00935769]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.]]), 'W2': array([[-0.00267888,  0.00530355, -0.00691661, -0.00396754],
       [-0.00687173, -0.00845206, -0.00671246, -0.00012665],
       [-0.0111731 ,  0.00234416,  0.01659802,  0.00742044]]), 'b2': array([[0.],
       [0.],
       [0.]])}


In [8]:
def linear_forward(A, W, b):
    
    Z = np.dot(W, A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    
    cache = (A, W, b)
    return Z, cache

In [9]:
A, W, b = linear_forward_test_case()
Z, cache = linear_forward(A, W, b)
print(Z)
print(cache)

[[ 3.26295337 -1.23429987]]
(array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ]]), array([[ 1.74481176, -0.7612069 ,  0.3190391 ]]), array([[-0.24937038]]))


In [10]:
def linear_forward_activation(A_pre, W, b, activation):
    
    Z, linear_cache = linear_forward(A_pre, W, b)
    if activation == 'sigmoid':
        A, activation_cache = sigmoid(Z)
    
    elif activation == 'relu':
        A, activation_cache = relu(Z)
    
    assert(A.shape == (W.shape[0], A_pre.shape[1]))    
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [11]:
A_pre, W, b = linear_activation_forward_test_case()

A, cache = linear_forward_activation(A_pre, W, b, 'sigmoid')
print(A)

A, cache = linear_forward_activation(A_pre, W, b, 'relu')
print(A)

[[0.96890023 0.11013289]]
[[3.43896131 0.        ]]


In [12]:
def L_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        A_pre = A
        A, cache = linear_forward_activation(A_pre, parameters["W" + str(l)], parameters["b" + str(l)], "relu")
        caches.append(cache)
        
    AL, cache = linear_forward_activation(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1, X.shape[1]))
    return AL, caches

In [13]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))


AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


In [14]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation ==  'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_pre, dW, db = linear_backward(dZ, linear_cache)
        
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_pre, dW, db == linear_backward(dZ, linear_cache)
        
    return dA_pre, dW, db

In [15]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))


NameError: name 'linear_backward' is not defined